In [23]:
#Importing required libraries.

import pandas as pd
import os
import string
from collections import Counter
import sklearn
import nltk
from natsort import natsorted
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score
from nltk.corpus import stopwords
from nltk import word_tokenize
import PySimpleGUI as sg

This code is used to open existing RAW article and filter it. We are removing stop words, and splitting text into list of words.

In [24]:
def articleClean(file):
    stop_words = set((stopwords.words('english')) + list(string.punctuation))

    symbols = ["\n", "’", "“", "”", "'", "`", "-", "[", "]", "»", ":"] 

    article = ""
    
    for i in symbols:
        file = file.replace(i, " ")

    def listToString(s): #function that converts arr of strings into one string (used later)
        str1 = " "
        return (str1.join(s))
    
    word_tokens= word_tokenize(file.lower()) #converts arcivle into array of words

    filtered_article = [] 

    for w in word_tokens: # check if word from article is the extra , if not assign the word in the clean words array
        if w not in stop_words and w.isalpha() == True:
            filtered_article.append(w)

    article = listToString(filtered_article)

    return article

Creating lists of articles and genres for training data.

In [25]:
all_articles = [] #training articles
genres = [] #genres

Store all technology articles in list and apply cleaning method.

In [26]:
os.chdir("/path/to/your/file") #access technology articles directory
#tech_res_art = []  
for i in range(1,101):
    tech_res = open("training_tech" + str(i) + ".txt", "r").read() #Read articles
    tech_res = articleClean(tech_res) #Apply cleaning to all articles
    #tech_res_art.append(tech_res)
    all_articles.append(tech_res) #Add articles into the list
    genres.append("technology") #Add 'technology' to genres

Store all sport articles in list and apply cleaning method.

In [27]:
os.chdir("/path/to/your/file")
sport_res_art = []

for i in range(1,101):
    sport_res = open("training_sport" + str(i) + ".txt", "r").read()
    sport_res = articleClean(sport_res)
    sport_res_art.append(sport_res)
    all_articles.append(sport_res)
    genres.append("sport")

Create DataFrame.

In [28]:
d = {'articles' : all_articles, 'genres' : genres}  
training_df = pd.DataFrame(data=d)  #Create training DataFrame
training_df

,articles,genres
0,european commission clarified claims perceived...,technology
1,audio player european union investigates micro...,technology
2,mentioned previously european commission embar...,technology
3,european commission responded reports perceive...,technology
4,god war ragnarok released days ago obviously s...,technology
...,...,...
195,dallas luka doncic admitted fatigue following ...,sport
196,portland trail blazers finished long six game ...,sport
197,nothing alleviates hangover nasty road trip li...,sport
198,arizona running back michael wiley right fends...,sport


Make everything lowercase.

In [29]:
training_df["articles"] = training_df["articles"].apply(lambda str : str.lower())
training_df

,articles,genres
0,european commission clarified claims perceived...,technology
1,audio player european union investigates micro...,technology
2,mentioned previously european commission embar...,technology
3,european commission responded reports perceive...,technology
4,god war ragnarok released days ago obviously s...,technology
...,...,...
195,dallas luka doncic admitted fatigue following ...,sport
196,portland trail blazers finished long six game ...,sport
197,nothing alleviates hangover nasty road trip li...,sport
198,arizona running back michael wiley right fends...,sport


In [30]:
training_df['label'] = training_df['genres'].apply(lambda x: 0 if x=="sport" else 1) #Create column 'label'. If genre is tech => label is 1, if sport => 0.
training_df  

,articles,genres,label
0,european commission clarified claims perceived...,technology,1
1,audio player european union investigates micro...,technology,1
2,mentioned previously european commission embar...,technology,1
3,european commission responded reports perceive...,technology,1
4,god war ragnarok released days ago obviously s...,technology,1
...,...,...,...
195,dallas luka doncic admitted fatigue following ...,sport,0
196,portland trail blazers finished long six game ...,sport,0
197,nothing alleviates hangover nasty road trip li...,sport,0
198,arizona running back michael wiley right fends...,sport,0


Create DataFrame for testing data.

In [31]:
test_articles = []
test_genres = []

In [32]:
os.chdir("/path/to/your/file")
for i in range(1, 101):
    test_tech_res = open("testing_tech" + str(i) + ".txt", "r").read()
    test_tech_res = articleClean(test_tech_res)
    test_articles.append(test_tech_res)
    test_genres.append("technology")

In [33]:
os.chdir("/path/to/your/file")
for i in range(1, 101):
    test_sport_res = open("testing_sport" + str(i) + ".txt", "r").read()
    test_sport_res = articleClean(test_sport_res)
    test_articles.append(test_sport_res)
    test_genres.append("sport")

In [34]:
d = {'articles' : test_articles, 'genres' : test_genres}
test_df = pd.DataFrame(data = d)
test_df

,articles,genres
0,might already seen pokémon scarlet violet come...,technology
1,battle larry normal type gym leader medali vic...,technology
2,son desperately wanted try game give bought di...,technology
3,something look forward interview pc world chie...,technology
4,powercolor teased next gen radeon rx xtx red d...,technology
...,...,...
195,eugene bo nix started week hobbled spectator s...,sport
196,estimated read time minutes eugene fourth pac ...,sport
197,red hot vegas golden knights edmonton saturday...,sport
198,vegas golden knights lost third time last four...,sport


In [35]:
test_df['label'] = test_df['genres'].apply(lambda x: 0 if x=="sport" else 1)
test_df

,articles,genres,label
0,might already seen pokémon scarlet violet come...,technology,1
1,battle larry normal type gym leader medali vic...,technology,1
2,son desperately wanted try game give bought di...,technology,1
3,something look forward interview pc world chie...,technology,1
4,powercolor teased next gen radeon rx xtx red d...,technology,1
...,...,...,...
195,eugene bo nix started week hobbled spectator s...,sport,0
196,estimated read time minutes eugene fourth pac ...,sport,0
197,red hot vegas golden knights edmonton saturday...,sport,0
198,vegas golden knights lost third time last four...,sport,0


In [36]:
df = training_df 
df = training_df.append(test_df, ignore_index=True)#  
df

/tmp/ipykernel_5309/3196510612.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = training_df.append(test_df, ignore_index=True)#


,articles,genres,label
0,european commission clarified claims perceived...,technology,1
1,audio player european union investigates micro...,technology,1
2,mentioned previously european commission embar...,technology,1
3,european commission responded reports perceive...,technology,1
4,god war ragnarok released days ago obviously s...,technology,1
...,...,...,...
395,eugene bo nix started week hobbled spectator s...,sport,0
396,estimated read time minutes eugene fourth pac ...,sport,0
397,red hot vegas golden knights edmonton saturday...,sport,0
398,vegas golden knights lost third time last four...,sport,0


Calculate the probability of article being about sport or technology.

In [37]:
training_x, test_X, training_y, test_y = train_test_split(df['articles'], df['label'], random_state=1) #X - articles for testing and training y- labels for testing and training

In [38]:
count_vector = CountVectorizer(strip_accents="ascii") 
training_x_count_vector = count_vector.fit_transform(training_x) 
X_test_count_vector = count_vector.transform(test_X) 

In [ ]:
naive_bayes = MultinomialNB() #Implementing Naive Bayes.
naive_bayes.fit(training_x_count_vector, training_y) #Provide algorithm with training data.
predictions = naive_bayes.predict(X_test_count_vector) #Predict article topic. 0 stands for sport, and 1 for technology.
predictions

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0])

In [44]:
#All variables here can have output only between 0 and 1, where 1 is the best, and 0 is the worst.

accuracy ='Accuracy score: ' + str(accuracy_score(test_y, predictions))
recallScore = 'Recall score: '+ str(recall_score(test_y, predictions))  #Ability to find all positive samples.
percisionScore = 'Precision score: ' + str(precision_score(test_y, predictions)) #Ability of the classifier not to label as positive a sample that is negative.
accuracy

'Accuracy score: 1.0'

In [41]:
tech_test = 'Computer Apple Pokemon Instagram'
sport_test = 'Game score champion'
def predictArticle(article):

    classifier = 0 

    clean_article = articleClean(article) #Apply article cleaning.
    clean_article = [clean_article] #Create vector out of testing article.

    count_vector_article = count_vector.transform(clean_article) #Convert cleaned article into a list of features.

    naive_B = naive_bayes.predict(count_vector_article) #Predict genre.

    classifier = naive_B[0]

    return classifier
predictArticle(sport_test) #Prediction.

0

creating Main method for UI.

In [ ]:
layout = [
    [sg.Text('Please enter article number')],
    [sg.Text('Article', size=(15, 1)), sg.InputText(key="lp1")],
    [sg.Button("Submit"), sg.Quit()]
] #Create layout window, buttons, and elements.
window = sg.Window('Naive Bayes Predictor', layout) #Title of the window.
while True:

    event, values = window.read() #User Input

    if event in (sg.WINDOW_CLOSED, "Quit"): #If press :Quit" => Close the window.
        break
    elif event == "Submit":
        text = values['lp1']
        if text == '': #If user input is empty print Null String
            print('Null string')
        else:
            try:
                value = int(text) #Try to conver input to interger if possible
                final_output = ''
                article_number = test_df["articles"][value] #Get the article with specified number from DataFrame
                answer = predictArticle(article_number) #Execute method and store output
                if answer == 1: #If prediction is 1 => Technology
                    final_output = "It is a techology related article"
                elif answer == 0:  #If prediciton is 0 => Sports
                    final_output = "It is a sports related article"
                else: 
                    final_output = "Incorrect value or format"
                print(df["articles"][value]) #Print article
                sg.popup(final_output, title = "Prediction") #Invoke popup window.
                print(f'Integer: {value}') #Print article number
                print("################")
            except:
                print("Not Integer") # If input is not an interger.

window.close() #close the window.